# Federated Learning with PySyft: A Practical Demonstration

In this demonstration, we will explore the concept of Federated Learning using PySyft. We will simulate multiple clients that train a simple machine learning model on a dataset while keeping their data local. 

## Prerequisites
Make sure you have the following libraries installed:
- PySyft
- PyTorch
- NumPy
- Matplotlib

You can install them using pip:
```bash
pip install syft torch numpy matplotlib


# Step 1: Import Libraries
Let's start by importing the necessary libraries

In [ ]:

```python
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy
import matplotlib.pyplot as plt


## Step 2: Setting Up the Federated Learning Environment

We'll create a virtual environment with several clients. Each client will simulate a device that holds a subset of data.


In [ ]:
# Create a hook to enable PySyft functionalities
hook = sy.TorchHook(torch)

# Create two virtual workers (clients)
client_1 = sy.VirtualWorker(hook, id="client_1")
client_2 = sy.VirtualWorker(hook, id="client_2")


## Step 3: Creating a Simple Dataset

We'll create a synthetic dataset for a classification problem. In a real-world scenario, each client would have its own data.


In [ ]:
# Create a simple dataset
def create_data(num_samples):
    # Features: Random data
    x = np.random.rand(num_samples, 2).astype(np.float32)
    # Labels: XOR pattern
    y = (x[:, 0] > 0.5).astype(np.float32) ^ (x[:, 1] > 0.5).astype(np.float32)
    return torch.tensor(x), torch.tensor(y)

# Generate data for each client
x_client_1, y_client_1 = create_data(100)
x_client_2, y_client_2 = create_data(100)

# Send data to respective clients
x_client_1 = x_client_1.send(client_1)
y_client_1 = y_client_1.send(client_1)
x_client_2 = x_client_2.send(client_2)
y_client_2 = y_client_2.send(client_2)


## Step 4: Defining the Neural Network Model

We will define a simple feedforward neural network for the classification task.


In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 5)
        self.fc2 = nn.Linear(5, 1)
    
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Instantiate the model
model = SimpleNN()


## Step 5: Training the Model Locally on Each Client

Each client will train the model locally on its own dataset for a few epochs.


In [ ]:
def train_model_on_client(model, x, y, client, epochs=10):
    model.send(client)  # Send model to client
    
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        predictions = model(x).squeeze()
        loss = criterion(predictions, y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    return model.get()  # Get the model back to the server

# Train the model on both clients
model = train_model_on_client(model, x_client_1, y_client_1, client_1, epochs=10)
model = train_model_on_client(model, x_client_2, y_client_2, client_2, epochs=10)


## Step 6: Aggregating the Model Updates

After training on both clients, we need to aggregate the model weights.


In [ ]:
# Aggregating model weights from both clients
def aggregate_models(models):
    # Get the average of the weights from the client models
    avg_model = models[0]  # Start with the first model
    for model in models[1:]:
        for param1, param2 in zip(avg_model.parameters(), model.parameters()):
            param1.data += param2.data / len(models)
    return avg_model

# Simulate the aggregation
model_aggregated = aggregate_models([model])


## Step 7: Evaluating the Aggregated Model

Finally, let's evaluate the aggregated model on a synthetic test dataset.


In [ ]:
# Create a test dataset
x_test, y_test = create_data(200)

# Forward pass through the aggregated model
with torch.no_grad():
    predictions = model_aggregated(torch.tensor(x_test, dtype=torch.float32)).squeeze()
    predictions = (predictions > 0.5).float()

# Calculate accuracy
accuracy = (predictions == torch.tensor(y_test, dtype=torch.float32)).float().mean()
print(f"Accuracy of the aggregated model: {accuracy:.2f}")


## Step 8: Visualization

Let's visualize the data points and decision boundary.


In [ ]:
# Plotting function
def plot_decision_boundary(model, x, y):
    x_min, x_max = x[:, 0].min() - 0.1, x[:, 0].max() + 0.1
    y_min, y_max = x[:, 1].min() - 0.1, x[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    Z = model(torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32))
    Z = (Z > 0.5).float().reshape(xx.shape)

    plt.contourf(xx, yy, Z.detach().numpy(), alpha=0.8)
    plt.scatter(x[:, 0], x[:, 1], c=y, edgecolors='k')
    plt.title("Decision Boundary")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.show()

# Plot decision boundary
plot_decision_boundary(model_aggregated, x_test, y_test)


## Conclusion

In this demonstration, we successfully implemented a simple Federated Learning setup using PySyft. We trained a model on local datasets without sharing any sensitive data, illustrating the key benefits of Federated Learning for privacy.

Feel free to modify the code, experiment with different datasets, and explore how Federated Learning can be applied to various problems!
